In [3]:
import pandas as pd
from datetime import datetime
from bson.timestamp import Timestamp

In [4]:
from pymongo import MongoClient
conn = MongoClient("mongodb://192.168.0.9",port=27017)
checklogdb = conn.dsi.log

La problematique concerne uniquement l'entrée principale des agents.

Inclure le code de transformation des données brutes en checklog.csv

In [12]:
presences = pd.read_csv("data/checklog.csv")

def create_timestamp(line, col_date, col_time):
    datetime_str = f"{line[col_date]} {line[col_time]}"
    try:
        datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        datetime_obj = datetime.strptime(datetime_str, "%m/%d/%Y %H:%M:%S")

    return Timestamp(int(datetime_obj.timestamp()), 1).as_datetime()

def create_datetime(row):
    try:
        return datetime.strptime(row, "%Y-%m-%d")
    except ValueError:
        return datetime.strptime(row, "%m/%d/%Y")

presences["log_checkin"] = presences.apply(create_timestamp, args=("log_date","log_checkin",), axis=1)
presences["log_checkout"] = presences.apply(create_timestamp, args=("log_date","log_checkout",), axis=1)
presences["log_date"] = presences.log_date.map(lambda x: create_datetime(x))

presences.to_csv("data/out.csv")

In [16]:
#checklogdb.insert_many(presences.to_dict(orient="records"))

In [15]:
presences.head()

,log_date,log_checkin,log_checkout,log_member_id,log_member_name,log_count
0,2023-01-09,2023-01-09 08:07:00+00:00,2023-01-09 00:00:00+00:00,1080720A,ANGOUA Guy-Charles,0
1,2023-01-10,2023-01-10 08:10:00+00:00,2023-01-10 00:00:00+00:00,1080720A,ANGOUA Guy-Charles,0
2,2023-01-11,2023-01-11 08:02:00+00:00,2023-01-11 00:00:00+00:00,1080720A,ANGOUA Guy-Charles,0
3,2023-01-12,2023-01-12 08:10:00+00:00,2023-01-12 00:00:00+00:00,1080720A,ANGOUA Guy-Charles,0
4,2023-01-13,2023-01-13 08:20:00+00:00,2023-01-13 00:00:00+00:00,1080720A,ANGOUA Guy-Charles,0
